# Backtest the strategies

Use an LLM to go through and predict the buy/ sell/ hold recommendation for the company for the given date. Steps needed:

1. Load the LLM - use DeepSeek R1 Qwen model at 7B parameters first and try the quantised models next
2. Step through each data and each financial statement to get a result
3. Log the results in a file and save to S3 (will need a logging file to save to S3 and resume in case of kernel crash)
4. Need a backtesting framework to apply the results


## Load libraries needed

In [1]:
import json
import boto3
from s3fs import S3FileSystem
import os
import datetime

import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from huggingface_hub import login
import torch
from accelerate import Accelerator

import pandas as pd

from IPython.display import Markdown, display

from helper import get_s3_folder
import s3_model
import company_data
from s3_model import S3ModelHelper

In [2]:
import importlib
importlib.reload(company_data)
importlib.reload(s3_model)

<module 's3_model' from '/project/s3_model.py'>

## Load the LLM

Models to test:
- Qwen (Qwen/Qwen2.5-7B-Instruct)
- Llama (meta-llama/Llama-3.2-7B-Instruct)
- DeepSeek (deepseek-ai/DeepSeek-R1-Distill-Qwen-14B)

In [34]:
# Log into Huggingface

with open('pass.txt') as p:
    hf_login = p.read()
    
hf_login = hf_login[hf_login.find('=')+1:hf_login.find('\n')]
login(hf_login, add_to_git_credential=False)

In [35]:
# Set up Quantization 
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"

)

In [53]:
accelerator = Accelerator()

In [36]:
# Flag to download from Huggingface again or use stored model
USE_HF = False
USE_QUANTIZATION = False

model_id = "meta-llama/Llama-3.2-3B-Instruct"
model_id_s3 = 'llama'


if USE_HF:
   
    pipeline = transformers.pipeline(
        "text-generation",
        model=model_id,
        model_kwargs={"torch_dtype": torch.bfloat16},
        device_map="auto",
    )
    
    if USE_QUANTIZATION:
        model = AutoModelForCausalLM.from_pretrained(model_id, device_map='auto', quantization_config=quant_config)
    else:
        model = AutoModelForCausalLM.from_pretrained(model_id, device_map='auto', torch_dtype=torch.bfloat16)
    tokenizer = AutoTokenizer.from_pretrained(model_id)
else:
    model_helper = s3_model.S3ModelHelper(s3_sub_folder='tmp/fs')
    if USE_QUANTIZATION:
        model = model_helper.load_model(model_id_s3, quant_config)
    else:
        model = model_helper.load_model(model_id_s3)
    tokenizer = AutoTokenizer.from_pretrained(model_id)

    pipeline = transformers.pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        model_kwargs={"torch_dtype": torch.bfloat16},
        device_map="auto",
    )
    model_helper.clear_folder(model_id_s3)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


## Load Financial PIT dataset

In [4]:
## Load from S3 using the helper file
sec_helper = company_data.SecurityData('tmp/fs','data_quarterly_pit.json')
all_data = sec_helper.get_all_data()

In [40]:
# USE WHILE DEVELOPING to
importlib.reload(company_data)
sec_helper = company_data.SecurityData('tmp/fs','data_quarterly_pit.json', all_data)

In [52]:
sec_helper.get_security_statement('2020-01-31','AON UN Equity','px')

,Price
Date,
2019-01-31,100.83
2019-02-28,101.25
2019-03-31,103.69
2019-04-30,118.13
2019-05-31,124.87
2019-06-30,127.68
2019-07-31,127.12
2019-08-31,129.44
2019-09-30,124.43


In [42]:
system_prompt = "You are a financial analyst and must make a buy, sell or hold decision on a company based only on the provided datasets. \
        Compute common financial ratios and then determine the buy or sell decision. Explain your reasons in less than 500 words. Provide a \
        confidence score for how confident you are of the decision. If you are not confident then lower the confidence score. \
        Provide your answer in JSON format like the two examples below: \
        {'Decision': BUY, 'confidence score': 80, 'Reason': 'Gross profit and EPS have both increased over time'} \
        {'Decision': SELL, 'confidence score': 90, 'Reason': 'Price has declined and EPS is falling'}"


In [43]:
prompt = sec_helper.get_prompt('2020-01-31','AON UN Equity', system_prompt)

In [45]:
tokens = tokenizer.apply_chat_template(prompt, tokenize=True, add_generation_prompt=True)
len(tokens)

4257

## Run an example in LLM

Run into out of memory problem - Potential fixes:
1. reduce size of model (quantize)
2. explore multi-gpu
3. reduce tokens

https://saturncloud.io/blog/how-to-solve-cuda-out-of-memory-error-in-pytorch/

https://huggingface.co/docs/accelerate/usage_guides/distributed_inference

https://medium.com/@geronimo7/llms-multi-gpu-inference-with-accelerate-5a8333e4c5db

Problem with splitting a single prompt into multiple gpus to calculate the result. Tensor parallelism - https://huggingface.co/docs/transformers/main/en/perf_train_gpu_many#tensor-parallelism

nvidia-smi will show available GPUs on the system.

In [47]:
def run_model(prompt, tokenizer, model):
    tokens = tokenizer.apply_chat_template(prompt, tokenize=False, add_generation_prompt=True)
    model_inputs = tokenizer([tokens], return_tensors='pt').to('cuda')
    generated_ids = model.generate(**model_inputs, max_new_tokens=1000)
    parsed_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    return tokenizer.batch_decode(parsed_ids, skip_special_tokens=True)[0]



In [49]:
# Time the execution
start_time = datetime.datetime.now()

# Run the model
response = run_model(prompt, tokenizer, model)

#Print the length of time to run
end_time = datetime.datetime.now()
print("Time to execute: ", end_time - start_time)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Time to execute:  0:00:26.567057


In [50]:
display(Markdown(response))

Here is the analysis of the company's financial statements:

**Income Statement Analysis**

1. Revenue: The revenue has been increasing over the past few years, with a steady growth rate of around 10-15%.
2. Gross Profit: The gross profit has also been increasing over the past few years, with a growth rate of around 20-25%.
3. Operating Expenses: The operating expenses have been relatively stable over the past few years, with a slight decrease in the last two years.
4. Operating Income: The operating income has been increasing over the past few years, with a growth rate of around 20-25%.
5. Non-Operating Income/ Loss: The non-operating income has been increasing over the past few years, with a growth rate of around 50-100%.
6. Net Interest Expense: The net interest expense has been decreasing over the past few years, with a growth rate of around 10-15%.
7. Income Tax Expense: The income tax expense has been increasing over the past few years, with a growth rate of around 20-25%.
8. Income from Continuing Operations: The income from continuing operations has been increasing over the past few years, with a growth rate of around 20-25%.

**Balance Sheet Analysis**

1. Current Assets: The current assets have been increasing over the past few years, with a growth rate of around 10-15%.
2. Non-Current Assets: The non-current assets have been increasing over the past few years, with a growth rate of around 10-15%.
3. Total Assets: The total assets have been increasing over the past few years, with a growth rate of around 10-15%.
4. Current Liabilities: The current liabilities have been increasing over the past few years, with a growth rate of around 10-15%.
5. Non-Current Liabilities: The non-current liabilities have been increasing over the past few years, with a growth rate of around 10-15%.
6. Total Liabilities: The total liabilities have been increasing over the past few years, with a growth rate of around 10-15%.
7. Share Capital & APIC: The share capital and additional paid-in capital have been increasing over the past few years, with a growth rate of around 10-15%.
8. Retained Earnings: The retained earnings have been increasing over the past few years, with a growth rate of around 10-15%.

**Financial Ratios**

1. Debt-to-Equity Ratio: The debt-to-equity ratio has been decreasing over the past few years, indicating a reduction in debt.
2. Current Ratio: The current ratio has been increasing over the past few years, indicating an improvement in liquidity.
3. Return on Equity (ROE): The ROE has been increasing over the past few years, indicating improved profitability.

**Buy, Sell, or Hold Decision**

Based on the analysis, the company's financial performance has been improving over the past few years, with increasing revenue, gross profit, operating income, and retained earnings. The debt-to-equity ratio and current ratio have also improved. However, the company's stock price has been volatile, with a decline in the last year.

**Confidence Score**: 80%

**Decision**: BUY

**Reason**: The company's financial performance has been improving, and the debt-to-equity ratio and current ratio have improved. However, the stock price has been volatile, and the company's profitability is not yet sustainable. A buy decision is recommended, but with caution.

In [30]:
def format_json(llm_output):
    form = llm_output['content'].replace('\n','')
    eoj = form.find('}```')
    additional = form[eoj + 4:]
    json_obj = json.loads(form[7:eoj + 1])
    json_obj['AdditionalContext'] = additional
    return json_obj

In [51]:
# start_time = datetime.datetime.now()
# #formatted_chat = tokenizer.apply_chat_template(prompt, tokenize=False, add_generation_prompt=True)
# outputs = pipeline(
#     prompt,
#     max_new_tokens=1000,
# )
# end_time = datetime.datetime.now()
# print("Time to execute: ", end_time - start_time)

# test_output = outputs[0]['generated_text'][-1]
# display(Markdown(test_output['content']))

In [28]:
df = sec_helper.get_security_statement('2020-01-31','AON UN Equity','is')